In [781]:
import tensorflow as tf 
import pandas as pd 
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.utils import pad_sequences

In [782]:
# get the data

def first_data(): 
    num_class = 5
    questions = []
    classes = []
    with open("./content.json", encoding='utf-8') as f: 
        # load json file
        data = json.load(f)
        for type_class in range(num_class): 
            # get the question data
            questions.append(data['intents'][type_class]['input'])
            # iterate for label class
            for i in range(len(data['intents'][type_class]['input'])): 
                # get the class data
                classes.append(data['intents'][type_class]['tag']) 
    # return all question and classes
    return questions, classes 

def final_data(questionsm, tags):
    final_questions  = []
    for row in range(len(questionsm)):
        for column in range(len(questionsm[row])): 
            final_questions.append(questionsm[row][column])
    #  define dataframe
    df = pd.DataFrame()
    # change the previou questions to finalq questions value 
    df['question'] = final_questions
    #  change the previou tags to final tag value 
    df['tag'] = tags
    # return final dataframe
    return  df


def encode_label(df: pd.DataFrame, tags: pd.Series): 
    le = LabelEncoder()
    le.fit(np.unique(tags))
    df['tag'] = le.fit_transform(df['tag'])
    return df, le.classes_

def get_train_test_data(df: pd.DataFrame): 
    # drp the tag colunns
    feature = np.array(df.drop('tag', axis =1))
    target = np.array(df['tag'])
    X_train, X_test, y_train, y_test =  train_test_split(feature, target, random_state = 42, test_size = 0.1)
    return  X_train, X_test, y_train, y_test

def text_preprocesing(text): 
    #  define stemmer from sastrawi 
    factory = StemmerFactory()
    # stemmer 
    stemmer = factory.create_stemmer()
    for row in range(len(text)): 
        for column in range(len(text[row])):
            text[row][column] = text[row][column].lower()
            # delete alfanumeric
            text[row][column] = re.sub(r'\W', ' ', text[row][column])
            # delete number
            text[row][column] = re.sub(r'\d+', '', text[row][column])
            # delete excessive whitespace
            text[row][column] = re.sub(r'\s+', ' ', text[row][column])
            text[row][column] = stemmer.stem(text[row][column])

    return text        
        

def create_model_and_train(x_train, y_train, x_test, y_test):
    # trying tokenize 
    x_train_list = x_train.tolist()
    x_test_list = x_test.tolist()
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token= "<OOV>")
    tokenizer.fit_on_texts(x_train_list)

    x_train_sequence = tokenizer.texts_to_sequences(x_train_list)
    x_train_padded = pad_sequences(x_train_sequence)

    x_test_sequence = tokenizer.texts_to_sequences(x_test_list)
    x_test_padded = pad_sequences(x_test_sequence)

    inputs = tf.keras.Input(shape=(None, ))
    x = tf.keras.layers.Embedding(1000, 2)(inputs)
    x = tf.keras.layers.LSTM(units = 200)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    predictions = tf.keras.layers.Dense(5, activation="softmax", name="predictions")(x)
    model = tf.keras.Model(inputs, predictions)
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer='adam',
        metrics=['accuracy']
    )

    history = model.fit(x_train_padded, y_train,epochs = 200 , validation_data = (x_test_padded, y_test))
    return model, history

# def predict


In [783]:
questions, classes = first_data()
final =  final_data(questions,  classes)

In [784]:
final.head(10)

,question,tag
0,Hai,greeting
1,Halo,greeting
2,Hei,greeting
3,Assalamualaikum,greeting
4,Hi,greeting
5,Apa kabar,greeting
6,Yo,greeting
7,Wassup,greeting
8,Hey,greeting
9,Selamat datang,greeting


In [785]:
df_final, class_encode = encode_label(final, final['tag'])
df_final.head(20)

,question,tag
0,Hai,2
1,Halo,2
2,Hei,2
3,Assalamualaikum,2
4,Hi,2
5,Apa kabar,2
6,Yo,2
7,Wassup,2
8,Hey,2
9,Selamat datang,2


In [786]:
class_encode.shape

(5,)

In [787]:
class_encode

array(['general', 'goodbye', 'greeting', 'izin', 'sakit'], dtype=object)

In [788]:
X_train, X_test, y_train, y_test = get_train_test_data(df_final)

In [789]:
X_train = text_preprocesing(X_train)
X_test = text_preprocesing(X_test)


In [790]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [791]:
create_model_and_train(X_train, y_train, X_test, y_test)
# create_model_and_train(df=df_final)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.1979 - loss: 1.6100 - val_accuracy: 0.1600 - val_loss: 1.6099
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1603 - loss: 1.6100 - val_accuracy: 0.2400 - val_loss: 1.6092
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2417 - loss: 1.6091 - val_accuracy: 0.2400 - val_loss: 1.6099
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2372 - loss: 1.6092 - val_accuracy: 0.1600 - val_loss: 1.6101
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2269 - loss: 1.6095 - val_accuracy: 0.2400 - val_loss: 1.6091
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2906 - loss: 1.6080 - val_accuracy: 0.2400 - val_loss: 1.6088
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2898 - loss: 1.6065 - val_accuracy: 0.1600 - val_loss: 1.6089
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2302 - loss: 1.6049 - val_accuracy: 0.1600 - val_loss: 1.6096

(<Functional name=functional_63, built=True>,
 <keras.src.callbacks.history.History at 0x22eab45cb90>)